<div>

> **Note**
>
> Code chunks run Python commands unless it starts with `%%bash`, in
> which case, those chunks run shell commands.

</div>

Partly following this PAGA
[tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/paga-paul15.html)
with some modifications.

## Loading libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import cellrank as cr

import scipy
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action="ignore", category=Warning)

# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(5, 5), facecolor='white', color_map = 'viridis_r') 

cr.settings.verbosity = 2

## Preparing data

In order to speed up the computations during the exercises, we will be
using a subset of a bone marrow dataset (originally containing about
100K cells). The bone marrow is the source of adult immune cells, and
contains virtually all differentiation stages of cell from the immune
system which later circulate in the blood to all other organs.

![](../figs/hematopoiesis.png)

If you have been using the **Seurat**, **Bioconductor** or **Scanpy**
toolkits with your own data, you need to reach to the point where can
find get:

-   A dimensionality reduction where to perform the trajectory (for
    example: PCA, ICA, MNN, harmony, Diffusion Maps, UMAP)
-   The cell clustering information (for example: from Louvain, k-means)
-   A KNN/SNN graph (this is useful to inspect and sanity-check your
    trajectories)

In this case, all the data has been preprocessed with Seurat with
standard pipelines. In addition there was some manual filtering done to
remove clusters that are disconnected and cells that are hard to
cluster, which can be seen in this
[script](https://github.com/NBISweden/workshop-scRNAseq/blob/master/scripts/data_processing/slingshot_preprocessing.Rmd)

The file trajectory_scanpy_filtered.h5ad was converted from the Seurat
object using the SeuratDisk package. For more information on how it was
done, have a look at the script:
[convert_to_h5ad.R](https://github.com/NBISweden/workshop-scRNAseq/blob/master/scripts/data_processing/convert_to_h5ad.R)
in the github repo.

You can download the data with the commands:

In [ ]:
import os
import urllib.request

# download pre-computed data if missing or long compute
fetch_data = True

# url for source and intermediate data
path_data = "https://export.uppmax.uu.se/naiss2023-23-3/workshops/workshop-scrnaseq"

path_results = "data/trajectory"
if not os.path.exists(path_results):
    os.makedirs(path_results, exist_ok=True)

path_file = "data/trajectory/trajectory_seurat_filtered.h5ad"
if not os.path.exists(path_file):
    file_url = os.path.join(
        path_data, "trajectory/trajectory_seurat_filtered.h5ad")
    urllib.request.urlretrieve(file_url, path_file)

## Reading data

We already have pre-computed and subsetted the dataset (with 6688 cells
and 3585 genes) following the analysis steps in this course. We then
saved the objects, so you can use common tools to open and start to work
with them (either in R or Python).

In [ ]:
adata = sc.read_h5ad("data/trajectory/trajectory_seurat_filtered.h5ad")
adata.var

In [ ]:
# check what you have in the X matrix, should be lognormalized counts.
print(adata.X[:10,:10])

## Explore the data

There is a umap and clusters provided with the object, first plot some
information from the previous analysis onto the umap.

In [ ]:
sc.pl.umap(adata, color = ['clusters','dataset','batches','Phase'],legend_loc = 'on data', legend_fontsize = 'xx-small', ncols = 2)

It is crucial that you performing analysis of a dataset understands what
is going on, what are the clusters you see in your data and most
importantly How are the clusters related to each other?. Well, let's
explore the data a bit. With the help of this table, write down which
cluster numbers in your dataset express these key markers.

  Marker    Cell Type
  --------- ----------------------------
  Cd34      HSC progenitor
  Ms4a1     B cell lineage
  Cd3e      T cell lineage
  Ltf       Granulocyte lineage
  Cst3      Monocyte lineage
  Mcpt8     Mast Cell lineage
  Alas2     RBC lineage
  Siglech   Dendritic cell lineage
  C1qc      Macrophage cell lineage
  Pf4       Megakaryocyte cell lineage

In [ ]:
markers = ["Cd34","Alas2","Pf4","Mcpt8","Ltf","Cst3", "Siglech", "C1qc", "Ms4a1", "Cd3e", ]
sc.pl.umap(adata, color = markers, use_raw = False, ncols = 4)

## Rerun analysis in Scanpy

Redo clustering and umap using the basic Scanpy pipeline. Use the
provided "X_harmony_Phase" dimensionality reduction as the staring
point.

In [ ]:
# first, store the old umap with a new name so it is not overwritten
adata.obsm['X_umap_old'] = adata.obsm['X_umap']

sc.pp.neighbors(adata, n_pcs = 30, n_neighbors = 20, use_rep="X_harmony_Phase")
sc.tl.umap(adata, min_dist=0.4, spread=3)

In [ ]:
#sc.tl.umap(adata, min_dist=0.6, spread=1.5)
sc.pl.umap(adata, color = ['clusters'],legend_loc = 'on data', legend_fontsize = 'xx-small', edges = True)

sc.pl.umap(adata, color = markers, use_raw = False, ncols = 4)

# Redo clustering as well
sc.tl.leiden(adata, key_added = "leiden_1.0", resolution = 1.0) # default resolution in 1.0
sc.tl.leiden(adata, key_added = "leiden_1.2", resolution = 1.2) # default resolution in 1.0
sc.tl.leiden(adata, key_added = "leiden_1.4", resolution = 1.4) # default resolution in 1.0

#sc.tl.louvain(adata, key_added = "leiden_1.0") # default resolution in 1.0
sc.pl.umap(adata, color = ['leiden_1.0', 'leiden_1.2', 'leiden_1.4','clusters'],legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)

In [ ]:
#Rename clusters with really clear markers, the rest are left unlabelled.

annot = pd.DataFrame(adata.obs['leiden_1.4'].astype('string'))
annot[annot['leiden_1.4'] == '10'] = '10_megakaryo' #Pf4
annot[annot['leiden_1.4'] == '17'] = '17_macro'  #C1qc
annot[annot['leiden_1.4'] == '11'] = '11_eryth' #Alas2
annot[annot['leiden_1.4'] == '18'] = '18_dend' #Siglech
annot[annot['leiden_1.4'] == '13'] = '13_mast' #Mcpt8
annot[annot['leiden_1.4'] == '0'] = '0_mono' #Cts3
annot[annot['leiden_1.4'] == '1'] = '1_gran' #Ltf
annot[annot['leiden_1.4'] == '9'] = '9_gran'
annot[annot['leiden_1.4'] == '14'] = '14_TC' #Cd3e
annot[annot['leiden_1.4'] == '16'] = '16_BC' #Ms4a1
annot[annot['leiden_1.4'] == '8'] = '8_progen'  # Cd34
annot[annot['leiden_1.4'] == '4'] = '4_progen' 
annot[annot['leiden_1.4'] == '5'] = '5_progen'

adata.obs['annot']=annot['leiden_1.4'].astype('category')

sc.pl.umap(adata, color = 'annot',legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)

annot.value_counts()
#type(annot)

# astype('category')

In [ ]:
# plot onto the Seurat embedding:
sc.pl.embedding(adata, basis='X_umap_old', color = 'annot',legend_loc = 'on data', legend_fontsize = 'xx-small', ncols =2)

## Run PAGA

Use the clusters from leiden clustering with leiden_1.4 and run PAGA.
First we create the graph and initialize the positions using the umap.

In [ ]:
# use the umap to initialize the graph layout.
sc.tl.draw_graph(adata, init_pos='X_umap')
sc.pl.draw_graph(adata, color='annot', legend_loc='on data', legend_fontsize = 'xx-small')
sc.tl.paga(adata, groups='annot')
sc.pl.paga(adata, color='annot', edge_width_scale = 0.3)

As you can see, we have edges between many clusters that we know are are
unrelated, so we may need to clean up the data a bit more.

## Filtering graph edges

First, lets explore the graph a bit. So we plot the umap with the graph
connections on top.

In [ ]:
sc.pl.umap(adata, edges=True, color = 'annot', legend_loc= 'on data', legend_fontsize= 'xx-small')

We have many edges in the graph between unrelated clusters, so lets try
with fewer neighbors.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=5,  use_rep = 'X_harmony_Phase', n_pcs = 30)
sc.pl.umap(adata, edges=True, color = 'annot', legend_loc= 'on data', legend_fontsize= 'xx-small')

### Rerun PAGA again on the data

In [ ]:
sc.tl.draw_graph(adata, init_pos='X_umap')
sc.pl.draw_graph(adata, color='annot', legend_loc='on data', legend_fontsize = 'xx-small')

In [ ]:
sc.tl.paga(adata, groups='annot')
sc.pl.paga(adata, color='annot', edge_width_scale = 0.3)

## Embedding using PAGA-initialization

We can now redraw the graph using another starting position from the
paga layout. The following is just as well possible for a UMAP.

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

Now we can see all marker genes also at single-cell resolution in a
meaningful layout.

In [ ]:
sc.pl.draw_graph(adata, color=['annot'], legend_loc='on data', legend_fontsize=  'xx-small')

Compare the 2 graphs

In [ ]:
sc.pl.paga_compare(
    adata, threshold=0.03, title='', right_margin=0.2, size=10, edge_width_scale=0.5,
    legend_fontsize=12, fontsize=12, frameon=False, edges=True)

## 

Choose a root cell for diffusion pseudotime. We have 3 progenitor
clusters, but cluster 5 seems the most clear.

In [ ]:
adata.uns['iroot'] = np.flatnonzero(adata.obs['annot']  == '5_progen')[0]

sc.tl.dpt(adata)

In [ ]:
sc.pl.draw_graph(adata, color=['annot', 'dpt_pseudotime'], legend_loc='on data', legend_fontsize= 'x-small')

## Cellrank

Error from multiprocessing/spawn.py in vscode/.qmd, but works fine to
run in terminal python.

In [ ]:
import cellrank as cr
from cellrank.kernels import PseudotimeKernel

#sc.pl.embedding(adata, basis="X_draw_graph_fa", color=["annot", "dpt_pseudotime"], legend_loc = "on data", legend_fontsize= 'x-small')
pk = PseudotimeKernel(adata, time_key="dpt_pseudotime")
pk

# calculates transition matrix based on:
#            self.connectivities,
#            self.pseudotime,

# adata.obsp['connectivities']      is same as pk.connectivities      

In [ ]:
pk.compute_transition_matrix(n_jobs=1)

pk


pk.plot_random_walks(
    seed=0,
    n_sims=100,
    start_ixs={"annot": "8_progen"},
    basis="umap",
    legend_loc="right",
    dpi=150,
)

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(pk)
print(g)


g.fit(n_states=10, cluster_key="annot")
g.plot_macrostates(which="all")


g.predict_terminal_states(method="top_n", n_states=4)
g.plot_macrostates(which="terminal")

g.compute_fate_probabilities()
g.plot_fate_probabilities(legend_loc="right")


cr.pl.circular_projection(adata, keys="annot", legend_loc="right")

In [ ]:
gran_drivers = g.compute_lineage_drivers(lineages="11_eryth_1")
#gran_drivers = g.compute_lineage_drivers(lineages="1_gran")
#KeyError: "Invalid lineage name `'1_gran'`. Valid names are: `['50', '14', '36', '5', '49', '23']`."

gran_drivers.head(10)



sc.pl.embedding(adata, basis="umap",color = list(gran_drivers.head(4).index),legend_loc = 'on data', legend_fontsize = 'xx-small', ncols = 2, use_raw = False)


sc.pl.embedding(adata, basis='X_draw_graph_fa',color = list(gran_drivers.head(4).index),legend_loc = 'on data', legend_fontsize = 'xx-small', ncols = 2, use_raw = False)


# use the ='X_draw_graph_fa' for running the gene estimates.

Same error as with our data, seems to be related to multiprocessing.
n_jobs=1 does not fix it.

Works fine to run the code from terminal, but not from quarto doc in
vscode! Why??

## Gene changes

We can reconstruct gene changes along PAGA paths for a given set of
genes

Choose a root cell for diffusion pseudotime. We have 3 progenitor
clusters, but cluster 5 seems the most clear.

In [ ]:
adata.uns['iroot'] = np.flatnonzero(adata.obs['annot']  == '5_progen')[0]

sc.tl.dpt(adata)

Use the full raw data for visualization.

In [ ]:
sc.pl.draw_graph(adata, color=['annot', 'dpt_pseudotime'], legend_loc='on data', legend_fontsize= 'x-small')

By looking at the different know lineages and the layout of the graph we
define manually some paths to the graph that corresponds to specific
lineages.

In [ ]:
# Define paths

paths = [('erythrocytes', ['5_progen', '8_progen', '6', '3', '7', '11_eryth']),
         ('lympoid', ['5_progen', '12', '16_BC', '14_TC']),
         ('granulo', ['5_progen', '4_progen', '2', '9_gran', '1_gran']),
         ('mono', ['5_progen', '4_progen', '0_mono', '18_dend', '17_macro'])
         ]

adata.obs['distance'] = adata.obs['dpt_pseudotime']

Then we select some genes that can vary in the lineages and plot onto
the paths.

In [ ]:
gene_names = ['Gata2', 'Gata1', 'Klf1', 'Epor', 'Hba-a2',  # erythroid
              'Elane', 'Cebpe', 'Gfi1',                    # neutrophil
              'Irf8', 'Csf1r', 'Ctsg',                     # monocyte
              'Itga2b','Prss34','Cma1','Procr',            # Megakaryo,Basophil,Mast,HPC
              'C1qc','Siglech','Ms4a1','Cd3e','Cd34']

In [ ]:
_, axs = pl.subplots(ncols=4, figsize=(10, 4), gridspec_kw={
                     'wspace': 0.05, 'left': 0.12})
pl.subplots_adjust(left=0.05, right=0.98, top=0.82, bottom=0.2)
for ipath, (descr, path) in enumerate(paths):
    _, data = sc.pl.paga_path(
        adata, path, gene_names,
        show_node_names=False,
        ax=axs[ipath],
        ytick_fontsize=12,
        left_margin=0.15,
        n_avg=50,
        annotations=['distance'],
        show_yticks=True if ipath == 0 else False,
        show_colorbar=False,
        color_map='Greys',
        groups_key='annot',
        color_maps_annotations={'distance': 'viridis'},
        title='{} path'.format(descr),
        return_data=True,
        use_raw=False,
        show=False)
    data.to_csv('data/trajectory/paga_path_{}.csv'.format(descr))
pl.savefig('data/trajectory/paga_path.pdf')
pl.show()

<div>

> **Discuss**
>
> As you can see, we can manipulate the trajectory quite a bit by
> selecting different number of neighbors, components etc. to fit with
> our assumptions on the development of these celltypes.
>
> Please explore further how you can tweak the trajectory. For instance,
> can you create a PAGA trajectory using the orignial umap from Seurat
> instead? Hint, you first need to compute the neighbors on the umap.

</div>

## Session info

```{=html}
<details>
```
```{=html}
<summary>
```
Click here
```{=html}
</summary>
```

In [ ]:
sc.logging.print_versions()

```{=html}
</details>
```